Filtering the feature matrix

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import time
import pytz
from datetime import datetime

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LinearRegression
import numpy.ma as ma


from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

/Users/joanbarreto/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Read the regions -----------

CircadianRegions = '../output/CircadianRegions_2kb.csv'
CircadianRegions = pd.read_csv(CircadianRegions)



#********************************************************
#
#           TESTING ONLY!!! (comment!)
#
CircadianRegions = CircadianRegions.sample(frac=0.1, replace=False, random_state=5461) # Sample a fraction of the df
#
#********************************************************



# Prepare data -----------
data_df = CircadianRegions[['gene','JTK_adjphase','region2kb']] # keep relevant cols
data_df = data_df.sort_values('gene') # make sure df is sorted by gene
# CircadianRegions = None




# Get features -----------

# Use an 8bp window and 3bp overlap
window_size = 8
overlap = 3
non_overlap = window_size-overlap
regions = data_df['region2kb']
# Slide window and get features
Features = regions.apply(lambda x: [x[i:i+window_size] for i in range(0, len(x)-7, non_overlap)])
Features.name = "features" # Rename
# Get unique features
unique_features = [item for sublist in Features for item in sublist]
unique_features = set(unique_features)



In [100]:
%%time

# One hot encoding -----------

# Create features df
features_df = pd.concat([data_df['gene'], Features], axis=1)
# data_df = None
# Set and fit MultiLabelBinarizer
mlb = MultiLabelBinarizer()
one_hot = mlb.fit_transform(features_df["features"]) # fit
# Create a new df with the one-hot encoding and the gene column
one_hot_df = pd.DataFrame(one_hot, columns=mlb.classes_)
one_hot_df["gene"] = features_df["gene"].values
one_hot_df = one_hot_df[["gene"] + list(mlb.classes_)] # "gene" col to front
one_hot_df = one_hot_df.set_index('gene') # Set gene as index
# Remove features containin "N" (only 6)
one_hot_df = one_hot_df.loc[:, ~one_hot_df.columns.str.contains('N')]

# Convert to numpy arrays
feature_names = np.array(one_hot_df.columns) # get feature names
gene_names = np.array(one_hot_df.index) # get gene names
np_data = one_hot_df.values  # Convert data to numpy array

print('computed one-hot-encoding')


computed one-hot-encoding
CPU times: user 3.37 s, sys: 4.58 s, total: 7.96 s
Wall time: 10.2 s


In [176]:
%%time

# Split data for training -----------
# Appears to be faster with a df than arrays

# Data for models
X_features = one_hot_df
Y_target = data_df["JTK_adjphase"]

# .to_numpy()

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X_features, Y_target, test_size=0.5, random_state=5461)

print('Finished splitting data')

Finished splitting data
CPU times: user 674 ms, sys: 683 ms, total: 1.36 s
Wall time: 1.52 s


In [218]:
%%time

# Select and train models -----------

# Create empty dfs to output
ModelsPerformance = pd.DataFrame()
FeatureImportance = pd.DataFrame()

# Create models
# knn = KNeighborsRegressor(n_neighbors=5)
# svr = SVR(kernel='rbf', C=1, gamma=0.1)
rfr = RandomForestRegressor(n_estimators=1, random_state=0)
# gbr = GradientBoostingRegressor(n_estimators=1, learning_rate=0.1, max_depth=1, random_state=0, loss='ls')
# Store them in dictionary
# Models = {'rfr':rfr,
#           'svr':svr,
#           'gbr':gbr,
#           'knn':knn}
Models = {'rfr':rfr}

# Select model
model_name = next(iter(Models)) # Name
model = next(iter(Models.values())) # Model

# Train the algorithm on the training set
start_time = time.time() # ***** Start timer *****
model.fit(X_train, y_train)

# Use the trained algorithm to predict the target variable of the test set
y_pred = model.predict(X_test)

# Feature importance
# if model_name == 'rfr' | model_name == 'gbr':
feature_importance = model.feature_importances_
# Create feature importance df for model
colname =  model_name + '_Importance'
FI_df = pd.DataFrame({'Feature': one_hot_df.columns, colname: feature_importance})
FI_df = FI_df.sort_values(by=[colname], ascending=False)
# Append to greater df
FeatureImportance = pd.concat([FeatureImportance,FI_df], axis=0) 


end_time = time.time() # ***** End timer *****
# Time to run
execution_time = round(end_time - start_time, 4)
print('Finished model ',model_name)

                  



Finished model  rfr
CPU times: user 23.2 s, sys: 1.25 s, total: 24.4 s
Wall time: 24.8 s


In [72]:
# Performance -----------

y_test = np.array(y_test, dtype=float)  # convert to np array
y_test=ma.masked_invalid(y_test)  # Treat nan
y_pred=ma.masked_invalid(y_pred)

PCC = round(np.corrcoef(y_test, y_pred)[0,1],2)
# y_pred = np.append(y_pred, np.nan)
# y_test = np.append(y_test, np.nan)

# np.any(np.isnan(y_test))
# np.any(np.isnan(y_pred))
# np.all(np.isfinite(y_test))
# np.all(np.isfinite(y_pred))

y_test[np.isfinite(y_test) == True] = 0
y_pred[np.isfinite(y_pred) == True] = 0
y_test[np.isnan(y_test) == True] = 0
y_pred[np.isnan(y_pred) == True] = 0

                  
# Get performance metrics into a df
mse = mean_squared_error(y_test, y_pred)
performance_df = pd.DataFrame({'Model':model_name,
                               'PCC':PCC,
                               'MSE':mse,
                               'ExecTime':execution_time},index=[0])
# Append to performance df
ModelsPerformance = pd.concat([ModelsPerformance,performance_df], axis=0)

In [ ]:
# Filter based on density -----------

# Copy data
df = one_hot_df
# Remove features containin "N" (only 6)
df = df.loc[:, ~df.columns.str.contains('N')]

feature_names = np.array(df.columns) # get feature names
gene_names = np.array(df.index) # get gene names
np_data.shape = df.values  # Convert data to numpy array


# Get densities
feature_densities = df.sum()
# feature_densities = df.sum() / df.shape[0]
feature_densities = feature_densities.sort_values(ascending=False)


# Filter out bottom and top thresholds (quantile-based)
# Calculate the nth percentile values
density_percentiles = np.percentile(feature_densities, [0,25,50,75,100])
print('Percentiles 0,25,50,75,100 are: \n', str(density_percentiles))
bottom_thresh = density_percentiles[1]
top_thresh = density_percentiles[2]
#filter
filtered_series = feature_densities[
    (feature_densities > bottom_thresh) & (feature_densities < top_thresh)]
filtered_features = filtered_series.index.to_list() # list names
filtered_features = df[filtered_features] # subset
print('Dimensions after density-based filter ',str(filtered_features.shape))




In [ ]:
# Filter based PCA -----------

df = filtered_features # copy data
print('Dimensions after density-based filter ',str(df.shape))
# standardize the data
scaler = StandardScaler()
X_std = scaler.fit_transform(df)
# Number of components (n_comp)
n_comp = 10
pc_names = ['PC{:02d}'.format(i) for i in range(1, n_comp+1)]
pca = PCA(n_components=n_comp)
pca.fit(X_std)
print('Fitted PCA')
# # transform the data to the new PCA space
# X_pca = pca.transform(X_std)
# # create a new dataframe with the PCA components
# df_pca = pd.DataFrame(X_pca, columns=pc_names)
# # add the original feature names as column names
# df_pca.columns = pc_names
# df_pca.index = df.index

# Get loadings
loadings = pca.components_
df_loadings = pd.DataFrame(loadings.T, columns=pc_names, index=df.columns)

# sort the rows (i.e., features) of the dataframe by their absolute loading value in descending order
df_loadings = df_loadings.apply(lambda x: x.abs().sort_values(ascending=False), axis=0)

# Empty list for selected features
selected_features=[]

for pc in pc_names:
    pc_column = pc # current PC
    pc_sorted = df_loadings[pc_column].sort_values(ascending=False) # sort loadings
    half_features = int(0.1 * len(pc_sorted.index)) # Select 10% of the features
    features_PC = list(pc_sorted[:half_features].index)
    selected_features.extend(features_PC)
    # len(selected_features)



In [ ]:
unique_features = np.unique(selected_features)
n_unique_features = len(unique_features) # get number of features
print('Total features: ', n_unique_features)
# Filtered dataset
filtered_df = one_hot_df.loc[:, unique_features]
# Export
filename = '../output/FilteredFeatures_PCA_n' + str(n_unique_features) + '.csv'
filtered_df.to_csv(filename)


# calculate elapsed time
elapsed_time = end_time - start_time

print("Elapsed time: ", round(elapsed_time,2), " seconds")